In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

print(f'GPU available: {torch.cuda.is_available()}')

Thu Apr 29 20:58:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install emoji

     |████████████████████████████████| 133kB 5.8MB/s 


In [16]:
import StockSentimentDataset
from torch.utils.data import DataLoader
from sklearn.utils.class_weight import compute_class_weight

train_ds, val_ds, test_ds, embeddings, featurizer = StockSentimentDataset.load_all_data()

class_weights = compute_class_weight(class_weight='balanced', classes=[0,1,2], y=train_ds.labels)
class_weights_tensor = torch.FloatTensor(class_weights)

In [44]:
import torch.nn.functional as f
class DAN(nn.Module):
    def __init__(self, layer_dims, embedding_dim, vocab_size, pad_token):
        super(DAN, self).__init__()

        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linears = nn.ModuleList([])
        self.pad_token = pad_token

        input_size = embedding_dim
        for hidden_size in layer_dims:
          self.linears.append(nn.Linear(input_size, hidden_size))
          input_size = hidden_size

        self.hidden_dropout = nn.Dropout(0.5)
        self.output_layer = nn.Linear(input_size, 3)

    def forward(self, X, train=False):
        X_mask = X != self.pad_token
        X_len = torch.sum(X_mask, dim=0)
        x = self.embeddings(X)
        x = torch.sum(x, dim=0) / X_len.view(X_len.size()[0], 1)

        for l in self.linears:
          x = l(x)
          x = f.relu(x)
          x = self.hidden_dropout(x)

        return self.output_layer(x)


In [76]:
from sklearn.metrics import f1_score
import random

def score_model(model, test_ds):
  model.eval()
  with torch.no_grad():
    test_dl = DataLoader(dataset=test_ds, batch_size=len(test_ds.samples),
      shuffle=False, collate_fn=StockSentimentDataset.collate_fn)
    predictions = []
    for x, y in test_dl:
      x = x.cuda()
      y = y.cuda()
      pred_scores = model(x)
      predictions.extend(torch.argmax(pred_scores, dim=1).tolist())
  return f1_score(test_ds.labels, predictions, labels=[0,1,2], average='macro')


In [74]:
#Training

from random import sample
import tqdm
import os
import subprocess
import random
from pretrained_embeddings import add_embeddings_to_embedding_matrix


def train(model, best_val_model, train_ds, val_ds, class_weights):
  class_weights = class_weights.cuda()
  optimizer = optim.Adam(model.parameters(), lr=0.001)
  loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)

  train_dl = DataLoader(dataset=train_ds, batch_size=len(train_ds.samples),
    shuffle=True, collate_fn=StockSentimentDataset.collate_fn)

  val_dl = DataLoader(dataset=val_ds, batch_size=len(val_ds.samples),
    shuffle=False, collate_fn=StockSentimentDataset.collate_fn)

  epoch_stats = []
  best_f1 = 0.0
  n_epochs_without_improvement = 0
  best_val_model.load_state_dict(model.state_dict())
  for epoch in range(250):
    totalLoss = 0.0
    n_samples = 0
    model.train()
    for x, y in tqdm.notebook.tqdm(train_dl, leave=False):
      x = x.cuda()
      y = y.cuda()
      model.zero_grad()
      predicted_scores = model(x)
      loss = loss_fn(predicted_scores, y)
      loss.backward()
      totalLoss += loss.item()
      n_samples += x.size()[1]
      optimizer.step()


    average_train_loss = totalLoss / float(n_samples)

    model.eval()
    total_val_loss = 0.0
    n_val_samples = 0
    with torch.no_grad():
      for x, y in val_dl:
        x = x.cuda()
        y = y.cuda()
        predicted_scores = model(x)
        loss = loss_fn(predicted_scores, y)
        total_val_loss += loss.item()
        n_val_samples += x.size()[1]
    
    average_val_loss = total_val_loss / float(n_val_samples)
    epoch_stats.append((epoch, average_train_loss, average_val_loss))
    val_f1 = score_model(model, val_ds)
    if epoch % 10 == 0:
      print("Epoch {0} finished. Train loss: {1}, Val loss: {2}".format(epoch, average_train_loss, average_val_loss))

    if val_f1 > best_f1:
      best_f1 = val_f1
      n_epochs_without_improvement = 0
      best_val_model.load_state_dict(model.state_dict())
    # else:
    #   n_epochs_without_improvement += 1
    #   if n_epochs_without_improvement == 10:
    #     break

  return best_f1, epoch_stats

model = DAN([120], 100, featurizer.vocab_size, featurizer.PADDING_ID)
best_val_model = DAN([120], 100, featurizer.vocab_size, featurizer.PADDING_ID)
add_embeddings_to_embedding_matrix(embeddings, model.embeddings, featurizer)
model = model.cuda()
best_val_model = best_val_model.cuda()
train(model, best_val_model, train_ds, val_ds, class_weights_tensor)

Epoch 0 finished. Train loss: 0.001338840448948154, Val loss: 0.005636105732041962


Epoch 10 finished. Train loss: 0.0012027757002575563, Val loss: 0.005531963036984814


Epoch 20 finished. Train loss: 0.0011719470501470619, Val loss: 0.00551126623640255


Epoch 30 finished. Train loss: 0.0011633599158560875, Val loss: 0.005478227625087816


Epoch 40 finished. Train loss: 0.0011525467546335017, Val loss: 0.00546535788750162


...

Epoch 210 finished. Train loss: 0.00014471561864133191, Val loss: 0.00597313472202846


Epoch 220 finished. Train loss: 0.00011880735162854588, Val loss: 0.006125455608173292


Epoch 230 finished. Train loss: 0.00010485206172280054, Val loss: 0.0064070991107395715


Epoch 240 finished. Train loss: 9.142760724255485e-05, Val loss: 0.006548522686471744


(0.5205415499533147,
 [(0, 0.001338840448948154, 0.005636105732041962),
  (1, 0.001274761849372956, 0.005627009333396445),
  (2, 0.0013171431779599163, 0.005599759062942193),
  (3, 0.0012803300641419733, 0.005574942851553158),
  (4, 0.0012557444923912862, 0.00556272207474222),
  (5, 0.0012334471095119767, 0.0055564337847184165),
  (6, 0.0012478061241678671, 0.005549514780239183),
  (7, 0.0012174109027724062, 0.00554385781288147),
  (8, 0.0012176047326183425, 0.005539208042378329),
  (9, 0.001203198244075964, 0.005535775301407794),
  (10, 0.0012027757002575563, 0.005531963036984814),
  (11, 0.001197371283511255, 0.005528589900659055),
  (12, 0.001196172108875774, 0.005525260555500887),
  (13, 0.001182263440424853, 0.005524205310004098),
  (14, 0.0011870979082466353, 0.0055218265981090316),
  (15, 0.0011852738356301757, 0.005519681439107778),
  (16, 0.0011817615548900776, 0.005518534962011844),
  (17, 0.0011852990151500807, 0.005515282859607618),
  (18, 0.0011805796911745312, 0.005514613

In [77]:
print(score_model(model, train_ds))
print(score_model(model, test_ds))
print(score_model(model, val_ds))

print(score_model(best_val_model, train_ds))
print(score_model(best_val_model, test_ds))
print(score_model(best_val_model, val_ds))


0.9989955210734432
0.5736700336700337
0.5064749827528333
0.98848053356802
0.5733897205745431
0.5205415499533147
